In [2]:
import panel as pn
import numpy as np
import param
import pandas as pd
from math import pi
from time import sleep

from bokeh.palettes import Category20c, Category20
from bokeh.plotting import figure, output_notebook
from bokeh.transform import cumsum

import requests
from urllib.parse import urlencode

pn.extension()

output_notebook()

VOIVODESHIPS = [
    'dolnośląskie',
    'kujawsko-pomorskie',
    'lubelskie',
    'lubuskie',
    'łódzkie',
    'małopolskie',
    'mazowieckie',
    'opolskie',
    'podkarpackie',
    'podlaskie',
    'pomorskie',
    'śląskie',
    'świętokrzyskie',
    'warmińsko-mazurskie',
    'wielkopolskie',
    'zachodniopomorskie'
]

CATEGORIES = {
    "---": "---",
    "5-gwiazdkowy (*****)":"KAT_5ST_",
    "4-gwiazdkowy (****)":"KAT_4ST_",
    "3-gwiazdkowy (***)":"KAT_3ST_",
    "2-gwiazdkowy (**)":"KAT_2ST_",
    "1-gwiazdkowy (*)":"KAT_1ST_",
}

KINDS = {
    '---':"---",
    'hotel':"HOT",
    'pensjonat':"PEN",
    'kemping':"KEM",
}

API_CWOH_URL = "https://api.turystyka.gov.pl/registers/open/cwoh"

def get_services():
    url = f"{API_CWOH_URL}/filters/questionnaires"
    result = requests.get(url)
    options = {}
    
    if result.status_code == 200:
        data = result.json()['content']
        options = {i['name']: i['key'] for i in  data}
    
    return options
    
def get_max_rooms():
    max_rooms = 200
    response = requests.get(
        f"{API_CWOH_URL}/filters/range/housingUnitsNumber"
    )
    if response.status_code == 200:
        max_rooms = response.json()['content']['housingUnitsNumber']                   
                           
    return max_rooms

max_rooms = get_max_rooms()
services = get_services()

srv_checkboxes = pn.widgets.CheckBoxGroup(name='Usługi/Wyposażenie', value=[], options=services)
cat_select = pn.widgets.Select(name='Kategoria', value="---", options=CATEGORIES, disabled=True)
kind_select = pn.widgets.Select(name='Rodzaj', value='---', options=KINDS)
rooms_slider = pn.widgets.IntRangeSlider(name='Liczba pokoi', start=1, end=max_rooms, value=(1, max_rooms), step=1)

def toggle_category_select(target, event):
    target.disabled = True if event.new == '---' else False
    if target.disabled:
        target.value = '---'

kind_select.link(cat_select, callbacks={'value': toggle_category_select})

@pn.depends(srv_checkboxes.param.value, cat_select.param.value, kind_select.param.value, rooms_slider.param.value)
def plot_cwoh(srv, cat, kind, room_range):
    data = {
        'woj': VOIVODESHIPS,
        'liczba': [],
        'l.p.': []
    }
    params = {}
    if srv:
        params['questionnaires'] = ','.join(srv)

    if kind != '---':
        params['kind'] = 'RODZ_' + kind
        if cat != '---':
            params['category'] = cat + kind
        
    if room_range:
        params['housingUnitsNumberMin'], params['housingUnitsNumberMax'] = room_range
    url = API_CWOH_URL
    for i,vs in enumerate(VOIVODESHIPS):
        params['voivodeship'] = vs
        url = "%s?%s" % (API_CWOH_URL,urlencode(params))
        result = requests.get(url)
        print(url)
        print(result.status_code)
        if result.status_code == 200:
            data['liczba'].append(result.json()['totalElements'])
            data['l.p.'].append(i+1)

    print(data)
            
    d = pd.DataFrame(data)
    d.set_index('l.p.', inplace=True)
    d['procent'] = d['liczba']/d['liczba'].sum()*100
    d.round(1)
    d['kąt'] = d['liczba']/d['liczba'].sum() * 2*pi
    d['kolor'] = Category20c[len(data['l.p.'])]
    
    p = figure(plot_height=440, title="Wyniki", toolbar_location=None,
           tools="hover", tooltips="@woj: @procent%", x_range=(-0.5, 1.0))    
    r = p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('kąt', include_zero=True), end_angle=cumsum('kąt'),
        line_color="white", fill_color='kolor', legend_field='woj', source=d)    
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None    
    
    df = pd.DataFrame(data).rename(columns={'woj':'województwo'})
    df.set_index('l.p.', inplace=True)
    
    return pn.Column(
        pn.pane.Bokeh(p),
        df,
        sizing_mode='stretch_both',
        width_policy='max',
        height_policy='max'         
    )  

dashboard = pn.Column(
    "## Obiekty hotelarskie w podziale na województwa",
    pn.Row(
        pn.Column(
            kind_select, 
            cat_select, 
            rooms_slider,
            srv_checkboxes,
            sizing_mode='stretch_height',            
        ),
        plot_cwoh
    ),
    sizing_mode='stretch_both',
    width_policy='max',
    height_policy='max'    
)


dashboard.servable()


Loading BokehJS ...

https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=dolno%C5%9Bl%C4%85skie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=kujawsko-pomorskie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=lubelskie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=lubuskie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=%C5%82%C3%B3dzkie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=ma%C5%82opolskie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNumberMax=200&voivodeship=mazowieckie
200
https://api.turystyka.gov.pl/registers/open/cwoh?housingUnitsNumberMin=1&housingUnitsNu

Column(height_policy='max', sizing_mode='stretch_both', width_policy='max')
    [0] Markdown(str)
    [1] Row
        [0] Column(sizing_mode='stretch_height')
            [0] Select(name='Rodzaj', options={'---': '---', ...}, value='---')
            [1] Select(disabled=True, name='Kategoria', options={'---': '---', ...}, value='---')
            [2] IntRangeSlider(end=200, name='Liczba pokoi', start=1, value=(1, 200))
            [3] CheckBoxGroup(name='Usługi/Wyposażenie', options={'Kręgielnia': 'questionna...})
        [1] ParamFunction(function)